# Why Chatbot
- reuse https://github.com/TensorMSA/tensormsa_jupyter/blob/master/chap13_chatbot_lecture/6.Chatbot(QA-seq2seq)-Word_onehot.ipynb
    - but, change `tf1` into `tf2`


## Seq2Seq를 활용한 간단한 Q/A 봇을 만들어보자
![이미지](http://suriyadeepan.github.io/img/seq2seq/seq2seq2.png)
* Python 3.5, Tensorflow 1.1, Konlpy (Mecab),Word2Vec (Gensim), matplotlib (Graph)

In [20]:
# -*- coding: utf-8 -*-
import tensorflow as tf
import numpy as np
import sys 

print (sys.version)
print (tf.__version__) #1.1이상 가능 

3.7.6 (default, Jan  8 2020, 13:42:34) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
2.3.1


### seq2seq를 위한 Data 구성
* 질의 응답별로 LIST로 구성
* operator사용 value값 기준 정렬

In [3]:
# 질문에 따른 답변 정의
train_data =  [
            ['안녕', '만나서 반가워']
                ,['넌누구니', '나는 AI 봇이란다.']
                 ,['내 멍멍이가 아파', '어제 밤부터']
                ,['지금 몇살이야?', '견종은 뭐야?']
            ]

from konlpy.tag import Mecab
mecab = Mecab('/usr/local/lib/mecab/dic/mecab-ko-dic')

In [7]:
train_data2 = list(map(lambda x : mecab.morphs(' '.join(x))  , train_data))
print(train_data2)

[['안녕', '만나', '서', '반가워'], ['넌', '누구', '니', '나', '는', 'AI', '봇', '이', '란다', '.'], ['내', '멍멍이', '가', '아파', '어제', '밤', '부터'], ['지금', '몇', '살', '이', '야', '?', '견', '종', '은', '뭐', '야', '?']]


In [8]:
import itertools
char_array = list(itertools.chain.from_iterable(train_data2))
    
char_array = ['P', '[', ']'] + list(set(char_array))  # Padding값을 0으로 주어 weight제외

max_input_text = max(len(s[0]) for s in train_data2)#입력의 차원 수
max_output_text = max(len(s[1]) for s in train_data2)#출력의 차원 수
print (char_array)

['P', '[', ']', '뭐', '니', '안녕', '란다', '?', '밤', '몇', '어제', '누구', '는', '부터', '서', '살', '가', '넌', '반가워', '만나', '내', '나', '종', '.', '아파', '야', '봇', '이', '멍멍이', '은', '견', 'AI', '지금']


In [9]:
train_data2 = []

for qna_data in train_data:
    train_data2 = train_data2 + list(map(lambda x : mecab.morphs(x) , qna_data))
                       
print (train_data2)

[['안녕'], ['만나', '서', '반가워'], ['넌', '누구', '니'], ['나', '는', 'AI', '봇', '이', '란다', '.'], ['내', '멍멍이', '가', '아파'], ['어제', '밤', '부터'], ['지금', '몇', '살', '이', '야', '?'], ['견', '종', '은', '뭐', '야', '?']]


In [10]:
max_input_text = 7
max_output_text = 7

# Vector 구성 (입력된 문장의 글자별 Vector)
 - 일반적으로 처리단위가 작아질수록 미등록어에서 자유롭고 작은 vector 차원을 유지할 수 있지만
 - 문장의 길이가 길어지고, 학습이 어려워지는 문제가 있기에 적절한 embedding을 찾아야하는데 
 - 이부분은 Biz Domain 별 차이가 있음 복잡도나 표현 가능성등을 적절한 균형에서 찾아야함 
 - 아래 소스는 이해하기 쉽도록 글자단위의 Onehot으로 구성

In [11]:
# enumerate 방법 사용 index : value 정렬
num_dic = {n: i for i, n in enumerate(char_array)}

dic_len = len(num_dic)

print ("# Char List : " + str(num_dic))
print ("# Char Size : " + str(dic_len))

# Char List : {'P': 0, '[': 1, ']': 2, '뭐': 3, '니': 4, '안녕': 5, '란다': 6, '?': 7, '밤': 8, '몇': 9, '어제': 10, '누구': 11, '는': 12, '부터': 13, '서': 14, '살': 15, '가': 16, '넌': 17, '반가워': 18, '만나': 19, '내': 20, '나': 21, '종': 22, '.': 23, '아파': 24, '야': 25, '봇': 26, '이': 27, '멍멍이': 28, '은': 29, '견': 30, 'AI': 31, '지금': 32}
# Char Size : 33


### One Hot Encodeing
* '안녕??'의 정렬하여 1의 값으로 정렬 <br>
안 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0] <br>
녕 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0] <br>
? [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] <br>

In [12]:
def make_train_data(train_data):
    input_batch = []
    output_batch = []
    target_batch = []

    for seq in train_data:
        # 인코더 셀의 입력값. 입력단어의 글자들을 한글자씩 떼어 배열로 만든다.
        seq_0 = mecab.morphs(seq[0]) 
        seq_1 = mecab.morphs(seq[1]) 
        input = [num_dic[n] for n in seq_0 + ['P'] * (max_input_text - len(seq_0)) ]# P는 Padding 값
        # 디코더 셀의 입력값. 시작을 나타내는 [ 심볼을 맨 앞에 붙여준다. (Seq의 구분)
        output = [num_dic[n] for n in (['['] + seq_1 + ['P'] * (max_output_text - len(seq_1)))]
        # 학습을 위해 비교할 디코더 셀의 출력값. 끝나는 것을 알려주기 위해 마지막에 ] 를 붙인다.
        target = [num_dic[n] for n in (seq_1 + ['P'] * (max_output_text - len(seq_1)) + [']'] )]
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)
    return input_batch, output_batch, target_batch

### 모델 저장을 위한 함수
* 현재폴더의 model폴더를 만들어 모델을 저장한다 
* 모델이 존재할 경우 삭제하고 새로 만든다

In [13]:
file_path = './model'
def model_file(file_path, flag):
    if(flag):
        import os
        saver = tf.train.Saver(tf.global_variables())

        if(not os.path.exists(file_path)):
            os.makedirs(file_path)
        saver.save(sess, ''.join(file_path + "/.model"))
        print("Model Saved")
    else:
        import shutil
        try:
            shutil.rmtree(file_path)
            print("Model Deleted")
        except OSError as e:
            if e.errno == 2:
                # 파일이나 디렉토리가 없음!
                print ('No such file or directory to remove')
                pass
            else:
                raise

### Tensorflow Graph 생성
* seq2seq모델의 Graph 생성
* 동일한 크기의 encoder과 decoder의 크기로 학습

In [18]:
from tensorflow.python.framework import ops

Instructions for updating:
non-resource variables are not supported in the long term


In [93]:
# 옵션 설정
learning_rate = 0.01
n_hidden = 128
total_epoch = 100
# one hot 위한 사이즈
n_class = n_input = dic_len

# 그래프 초기화 
ops.reset_default_graph()
# Seq2Seq 모델은 인코더의 입력과 디코더의 입력의 형식이 같다.
#enc_input = tf.placeholder(tf.float32, [None, None, n_input])
#dec_input = tf.placeholder(tf.float32, [None, None, n_input])
#targets = tf.placeholder(tf.int64, [None, None])

enc_input = tf.Variable(tf.ones(shape=( n_input, n_input, n_input)), name="enc_input", dtype=tf.dtypes.float32)
dec_input = tf.Variable(tf.ones(shape=(n_input, n_input, n_input)), name="dec_input", dtype=tf.dtypes.float32)
#enc_input = tf.keras.Input(shape=(None, None, n_input), dtype=tf.dtypes.float32)
#dec_input = tf.keras.Input(shape=(None, None, n_input), dtype=tf.dtypes.float32)
targets = tf.keras.Input(shape=(None, None), dtype=tf.dtypes.float32)

In [110]:
train_data

[['안녕', '만나서 반가워'],
 ['넌누구니', '나는 AI 봇이란다.'],
 ['내 멍멍이가 아파', '어제 밤부터'],
 ['지금 몇살이야?', '견종은 뭐야?']]

In [68]:
input_batch, output_batch, target_batch = make_train_data(train_data)

In [69]:
input_batch 

[array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [1., 0., 0., 0., 0., 0., 0., 0.,

In [70]:
output_batch

[array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [1., 0., 0., 0., 0., 0., 0., 0.,

In [71]:
target_batch

[[19, 14, 18, 0, 0, 0, 0, 2],
 [21, 12, 31, 26, 27, 6, 23, 2],
 [10, 8, 13, 0, 0, 0, 0, 2],
 [30, 22, 29, 3, 25, 7, 0, 2]]

In [102]:
# https://wdprogrammer.tistory.com/37

from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense, BatchNormalization

# a part of encoder
encoder_inputs = Input(shape=(None, n_input), name='encoder_input')
encoder = LSTM(n_hidden, return_sequences=True, return_state=True, name='encoder')
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

# a part of decoder
decoder_inputs = Input(shape=(None, n_input), name='decoder_input')
decoder_lstm = LSTM(n_hidden, return_sequences=True, return_state=True, name='decoder')
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
batchNorm = BatchNormalization()
decoder_dense = Dense(n_input, activation='softmax')
decoder_outputs = decoder_dense(batchNorm(decoder_outputs))

# a model to train
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [104]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, None, 33)]   0                                            
__________________________________________________________________________________________________
decoder_input (InputLayer)      [(None, None, 33)]   0                                            
__________________________________________________________________________________________________
encoder (LSTM)                  [(None, None, 128),  82944       encoder_input[0][0]              
__________________________________________________________________________________________________
decoder (LSTM)                  [(None, None, 128),  82944       decoder_input[0][0]              
                                                                 encoder[0][1]         

### 학습결과 출력
* matplotlib 활용 학습 결과 출력

In [106]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [152]:
len(input_batch)

4

In [153]:
# list to tensor
input_batch2 = []
output_batch2 = []
for i in input_batch: input_batch2.append(tf.convert_to_tensor(i))
for o in output_batch: output_batch2.append(tf.convert_to_tensor(o))

    
input_batch_t = tf.convert_to_tensor(input_batch2)
output_batch_t = tf.convert_to_tensor(output_batch2)

In [155]:
input_batch_t

<tf.Tensor 'packed:0' shape=(4, 7, 33) dtype=float64>

In [159]:
#{enc_input: input_batch,dec_input: output_batch, targets: target_batch}

hist = model.fit([enc_input, input_batch_t], epochs=50, batch_size=200, steps_per_epoch=4)
print('\nAccuracy: {:.4f}'.format(model.evaluate(X_validation, Y_validation)[1]))

Train on 4 samples
Epoch 1/50


FailedPreconditionError: Error while reading resource variable enc_input from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/enc_input/N10tensorflow3VarE does not exist.
	 [[{{node enc_input/Read/ReadVariableOp}}]]

In [ ]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper left')

plt.show()

### 예측 수행

In [ ]:
# 최적화가 끝난 뒤, 변수를 저장합니다.
model_file(file_path, True)

# 단어를 입력받아 번역 단어를 예측하고 디코딩하는 함수
def predict(word):
    input_batch, output_batch, target_batch = make_train_data([word])
    # 결과가 [batch size, time step, input] 으로 나오기 때문에,
    # 2번째 차원인 input 차원을 argmax 로 취해 가장 확률이 높은 글자를 예측 값으로 만든다.
    # http://pythonkim.tistory.com/73
    prediction = tf.argmax(model, 2)
    result = sess.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})
    # 결과 값인 숫자의 인덱스에 해당하는 글자를 가져와 글자 배열을 만든다.
    decoded = [char_array[i] for i in result[0]]
        
    if 'P' in decoded:
        end = decoded.index('P')
        decoded = decoded[:end]
    elif ']' in decoded:
        end = decoded.index(']')
        decoded = decoded[:end] 
    return decoded




print ("Q: 넌누구니")
print("A: " + ' '.join(predict(['넌누구니',''])))
print ("Q: 피자 주문 할께")
print("A: " + ' '.join(predict(['피자 주문 할께',''])))
print ("Q: 음료는 멀로")
print("A: " + ' '.join(predict(['음료는 멀로',''])))

model_file(file_path, False)